In [1]:
!pip install pillow
!KERAS_BACKEND=tensorflow python -c "from keras import backend"

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
/home/daniel/anaconda3/envs/rocmtf/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers

/home/daniel/anaconda3/envs/rocmtf/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# Input metadata
img_width, img_height = 150, 150
ds_train = "./data/train"
ds_test = "./data/test"

In [4]:
# Divide the data
data_gen = ImageDataGenerator(rescale=1./255)
gen_train = data_gen.flow_from_directory(
    ds_train, 
    target_size=(img_width, img_height), 
    batch_size=16, 
    class_mode="binary"
)
gen_test = data_gen.flow_from_directory(
    ds_test, 
    target_size=(img_width, img_height), 
    batch_size=32, 
    class_mode="binary"
)

def network_generators(train_dir, test_dir, img_w, img_h, train_batch, test_batch):
    data_generator = ImageDataGenerator(rescale=1./255)
    gen_train = data_generator.flow_from_directory(
        train_dir, 
        target_size=(img_w, img_h), 
        batch_size=train_batch, 
        class_mode="binary"
    )
    gen_test = data_gen.flow_from_directory(
        train_dir, 
        target_size=(img_w, img_h), 
        batch_size=test_batch, 
        class_mode="binary"
    )
    
    return (gen_train, gen_test)


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [5]:
#######################################
#### MODELS
#######################################

# Copy-paste model
def model_naive(img_w, img_h):
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(32, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(64, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model

# Larger model, with relu at the end
def model_larger(img_w, img_h):
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Convolution2D(64, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Convolution2D(64, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Convolution2D(64, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(32, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('relu'))
    
    return model

In [11]:
# model = model_naive(img_width, img_height) [0.4223675976256625, 0.8190354567307693]
# model = model_larger(img_width, img_height) [7.990353628993034, 0.0]
model = model_naive(img_width, img_height)
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

/home/daniel/anaconda3/envs/rocmtf/lib/python3.5/site-packages/ipykernel_launcher.py:32: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(150, 150,...)`
/home/daniel/anaconda3/envs/rocmtf/lib/python3.5/site-packages/ipykernel_launcher.py:36: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
/home/daniel/anaconda3/envs/rocmtf/lib/python3.5/site-packages/ipykernel_launcher.py:40: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`


In [12]:
nb_epoch = 30
nb_train_samples = 2048
nb_validation_samples = 832

In [13]:
model.fit_generator(
        gen_train,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=gen_test,
        nb_val_samples=nb_validation_samples)

/home/daniel/anaconda3/envs/rocmtf/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/home/daniel/anaconda3/envs/rocmtf/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=30, steps_per_epoch=128, validation_steps=832, validation_data=<keras_pre...)`
  


Epoch 1/30
128/128 [==============================] - 14s 107ms/step - loss: 7.9929 - acc: 0.0098 - val_loss: 7.9712 - val_acc: 0.0000e+00
Epoch 2/30
128/128 [==============================] - 13s 101ms/step - loss: 7.8469 - acc: 0.0283 - val_loss: 7.9712 - val_acc: 0.0000e+00
Epoch 3/30
128/128 [==============================] - 13s 100ms/step - loss: 7.9573 - acc: 9.7656e-04 - val_loss: 7.9712 - val_acc: 0.0000e+00
Epoch 4/30
128/128 [==============================] - 13s 103ms/step - loss: 7.7789 - acc: 0.0000e+00 - val_loss: 7.9712 - val_acc: 0.0000e+00
Epoch 5/30
128/128 [==============================] - 13s 102ms/step - loss: 7.9089 - acc: 4.8828e-04 - val_loss: 7.9712 - val_acc: 0.0000e+00
Epoch 6/30
128/128 [==============================] - 13s 103ms/step - loss: 8.1658 - acc: 0.0000e+00 - val_loss: 7.9712 - val_acc: 0.0000e+00
Epoch 7/30
128/128 [==============================] - 13s 105ms/step - loss: 8.0724 - acc: 4.8828e-04 - val_loss: 7.9712 - val_acc: 0.0000e+00
Epoch 8

In [14]:
model.save_weights("models/cnn_20_epochs.h5")
#model.save_weights("models/cnn_larger_20_epochs.h5")

In [15]:
model.evaluate_generator(gen_train, nb_validation_samples)

[7.990353628993034, 0.0]

In [40]:
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = model_naive(img_width, img_height)
model.load_weights("models/cnn_larger_20_epochs.h5")


img_path = './data/predict/dog_3.jpg'
img = image.load_img(img_path, target_size=(img_width, img_height))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', preds)

/home/daniel/anaconda3/envs/rocmtf/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(150, 150,...)`
  
/home/daniel/anaconda3/envs/rocmtf/lib/python3.5/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  if sys.path[0] == '':
/home/daniel/anaconda3/envs/rocmtf/lib/python3.5/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  app.launch_new_instance()


Predicted: [[1.]]


In [18]:
for x in range(1, 3):
    print(x)


1
2
